<a href="https://colab.research.google.com/github/DArsem2306/Programa-Python-Utec/blob/main/modulo2_Estructuras_de_Datos/Clase09/%5B7%5D_Actividad_41_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 41: Caso Trackus

## [0] Librerías necesarias para el proyecto

In [1]:
# Conectamos Google Drive con Google Colab
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Librerías necesarias para el desarrollo del proyecto
import pandas as pd
import numpy as np
from datetime import date

## [1] Entendimiento de los datos

Etapa inicial en un proyecto de Ciencia de Datos, nos permite explorar las características de los datos a ser modelados.

In [3]:
# Lectura de los datos (formato parquet)
ruta = "/gdrive/My Drive/UTEC/"
dataset = "dataset_test.parquet"
data = pd.read_parquet(ruta+dataset, engine='pyarrow')

In [4]:
# Dimensionalidad de los datos a nivel de número de filas y columnas
data.shape

(116211465, 5)

In [5]:
# Inspección de los datos, cinco registros iniciales
data.head(5)

,Date,Account_id,Product_id,Category,Quantity
0,2022-01-03,33236916,8350,Gaseosas,1
1,2022-01-03,33237477,15354,Maltas,5
2,2022-01-03,33229164,14016,Cervezas,25
3,2022-01-03,33229164,22270,Cervezas,2
4,2022-01-03,33229164,23760,Cervezas,21


In [6]:
data[['Quantity']].describe()

,Quantity
count,1.162115e+08
mean,1.438517e+01
std,9.367268e+01
min,0.000000e+00
25%,1.000000e+00
50%,2.000000e+00
75%,6.000000e+00
max,1.187600e+04


In [7]:
# Cantidad de clientes únicos
data.Account_id.nunique()

12618

In [8]:
# Cantidad de productos únicos
data.Product_id.nunique()

182

In [9]:
# Cantidad de categorías de productos únicos
data.Category.nunique()

7

In [10]:
# Evaluamos a las categorías
data.groupby(['Category'])['Account_id'].count()

,Account_id
Category,
Agua,1182090
Cervezas,37279999
Gaseosas,52033722
Jugos Mp,3320394
Lacteos,1649264
Licor Mp,578113
Maltas,20167883


In [11]:
data.Category.unique()

array(['Gaseosas', 'Maltas', 'Cervezas', 'Agua', 'Jugos Mp', 'Licor Mp',
       'Lacteos'], dtype=object)

In [12]:
# Inspeccionamos la compra del cliente (código 33236916)
data[data['Account_id']==33236916]

,Date,Account_id,Product_id,Category,Quantity
0,2022-01-03,33236916,8350,Gaseosas,1
2083,2022-01-03,33236916,8350,Gaseosas,1
3427,2022-01-03,33236916,8350,Gaseosas,1
5825,2022-01-03,33236916,8350,Gaseosas,1
7904,2022-01-03,33236916,8350,Gaseosas,1
...,...,...,...,...,...
116203108,2022-07-30,33236916,8228,Gaseosas,2
116203109,2022-07-30,33236916,34366,Cervezas,10
116205567,2022-07-30,33236916,8222,Gaseosas,3
116205568,2022-07-30,33236916,8228,Gaseosas,2


In [13]:
# Creamos una lista de campos que serán analizados
vars = data.columns.to_list()
vars

['Date', 'Account_id', 'Product_id', 'Category', 'Quantity']

In [14]:
# Agrupamos la data (está expresada a nivel transaccional) a nivel cliente - producto - día
data = data.groupby(vars[:-1])[vars[-1]].sum().reset_index()

In [15]:
# Inspeccionamos la compra del cliente (código 33236916)
data[data['Account_id']==33236916]

,Date,Account_id,Product_id,Category,Quantity
659,2022-01-03,33236916,8350,Gaseosas,89
35158,2022-01-08,33236916,8326,Gaseosas,89
35159,2022-01-08,33236916,9850,Gaseosas,178
35160,2022-01-08,33236916,9852,Gaseosas,178
35161,2022-01-08,33236916,12620,Gaseosas,267
...,...,...,...,...,...
1271391,2022-07-27,33236916,14014,Cervezas,5400
1271392,2022-07-27,33236916,34620,Jugos Mp,54
1288590,2022-07-30,33236916,8222,Gaseosas,153
1288591,2022-07-30,33236916,8228,Gaseosas,102


In [16]:
# Inspeccionamos la nueva dimensionalidad de los datos
data.shape

(1292772, 5)

In [17]:
# Inspeccionamos el tipo del dato
data.dtypes

,0
Date,object
Account_id,int64
Product_id,int64
Category,object
Quantity,int32


In [18]:
# Cambiamos el campo Date de objeto a fecha
data['Date'] = data['Date'].astype('datetime64[ns]')

In [19]:
# Calculamos la fecha mínima y máxima de transacción
print("Fecha Mínima: ", data.Date.min())
print("Fecha Máxima: ", data.Date.max())
print("*"*100)

Fecha Mínima:  2022-01-03 00:00:00
Fecha Máxima:  2022-07-31 00:00:00
****************************************************************************************************


In [20]:
# Generamos el resumen de la cantidad de clientes a nivel día
res_date = data.groupby(['Date'])['Account_id'].count().reset_index()

In [21]:
res_date.head(10)

,Date,Account_id
0,2022-01-03,1994
1,2022-01-04,6935
2,2022-01-05,8406
3,2022-01-06,9073
4,2022-01-07,6721
5,2022-01-08,7636
6,2022-01-11,4033
7,2022-01-12,11118
8,2022-01-13,11989
9,2022-01-14,8309


In [22]:
# Estadísticas del resumen de la cantidad de clientes a nivel día
res_date['Account_id'].describe().T

,Account_id
count,188.000000
mean,6876.446809
std,2251.632996
min,3.000000
25%,5975.250000
50%,7187.500000
75%,8394.000000
max,11989.000000


In [30]:
# Calculamos las fechas que presentan un comportamiento atípico a nivel de clientes
p75 = np.quantile(res_date['Account_id'],0.75)
p25 = np.quantile(res_date['Account_id'],0.25)
ric = p75 - p25
l_inf = p25 - (1.5*ric)
l_sup = p75 - (1.5*ric)

In [31]:
# Calculamos la fecha mínima y máxima de transacción
print("Umbral mínimo de clientes por día: ", l_inf)
print("Umbral máximo de clientes por día ", l_sup)
print("*"*100)

Umbral mínimo de clientes por día:  2347.125
Umbral máximo de clientes por día  4765.875
****************************************************************************************************


In [32]:
# Extraemos los campos Year y Month desde el campo Date
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

In [33]:
# Inspeccionamos los primeros cinco registros
data.head(5)

,Date,Account_id,Product_id,Category,Quantity,Year,Month
0,2022-01-03,33217794,8320,Gaseosas,267,2022,1
1,2022-01-03,33217794,8346,Gaseosas,89,2022,1
2,2022-01-03,33217794,9090,Gaseosas,356,2022,1
3,2022-01-03,33217995,16206,Cervezas,178,2022,1
5,2022-01-03,33218037,29554,Gaseosas,178,2022,1


In [35]:
# Guardamos los datos agrupados a nivel cliente - producto - día
ruta = "/gdrive/My Drive/UTEC/out/"
dataset = "data_agrupada.csv"
data.to_csv(ruta+dataset,header='True',index_label='False')

## [2] Preparación de los datos

Etapa en la que limpiamos los datos para simplificar la complejidad natural de la búsqueda de patrones por parte de los algoritmos.

In [37]:
# Calculamos los rangos intercuartílicos para la identificación de outliers a nivel de cantidad de compras

q3 = np.quantile(data['Quantity'],0.75)
q1 = np.quantile(data['Quantity'],0.25)
ric = q3 - q1
lim_sup = q3 + (1.5*ric)
lim_inf = q1 - (1.5*ric)

In [38]:
# Límites según criterio de Boxplot
lim_inf, lim_sup

(np.float64(-311.5), np.float64(756.5))

In [39]:
# Filtramos los casos que se encuentran contenidos entre los límites
data = data[(data['Quantity']>= lim_inf) & (data['Quantity']<= lim_sup)]

In [40]:
# Exploramos la nueva dimensionalidad de los datos, no están considerados los valores outliers
data.shape

(1013113, 7)

In [41]:
# Analizamos la cantidad promedio de productos comprados por día
data.groupby(['Month'])['Quantity'].mean()

,Quantity
Month,
1,201.935415
2,207.904629
3,212.586877
4,203.222061
5,206.731235
6,199.016403
7,200.304559


In [42]:
# Analizamos la mediana de la cantidad de productos comprados por día
data.groupby(['Month'])['Quantity'].median()

,Quantity
Month,
1,178.0
2,178.0
3,178.0
4,178.0
5,178.0
6,162.0
7,132.0


In [43]:
# Analizamos la dispersión de la cantidad de productos comprados por día
data.groupby(['Month'])['Quantity'].std()

,Quantity
Month,
1,153.817380
2,159.721616
3,159.772683
4,151.604388
5,155.211105
6,149.649880
7,175.793165
